In [1]:
import sys
sys.executable

'/Users/mushirsayed/Desktop/Football Stats/venv/bin/python3.12'

In [1]:
from pathlib import Path
import requests

API_KEY = Path("API KEY.txt").read_text().strip()

headers = {
    "Authorization": f"Bearer {API_KEY}",
    "User-Agent": "Mozilla/5.0"
}

url = "https://api.collegefootballdata.com/recruiting/players"
params = {"year": 2024}

r = requests.get(url, headers=headers, params=params, timeout=30)

print("status:", r.status_code)
print("first_200_chars:", r.text[:200])

status: 200
first_200_chars: [{"id":"108699","athleteId":"5079720","recruitType":"HighSchool","year":2024,"ranking":1,"name":"Jeremiah Smith","school":"Chaminade-Madonna","committedTo":"Ohio State","position":"WR","height":75,"we


In [3]:
import pandas as pd

data = r.json()
df = pd.DataFrame(data)

print(df.shape)
df.head()

(2548, 17)


,id,athleteId,recruitType,year,ranking,name,school,committedTo,position,height,weight,stars,rating,city,stateProvince,country,hometownInfo
0,108699,5079720,HighSchool,2024,1,Jeremiah Smith,Chaminade-Madonna,Ohio State,WR,75.0,214.0,5,0.9997,Opa Locka,FL,USA,"{'latitude': 25.896733849999997, 'longitude': ..."
1,108700,5079698,HighSchool,2024,2,Ellis Robinson IV,IMG Academy,Georgia,CB,73.0,185.0,5,0.9994,Bradenton,FL,USA,"{'latitude': 27.4989278, 'longitude': -82.5748..."
2,108701,5079376,HighSchool,2024,3,Cam Coleman,Central,Auburn,WR,75.0,180.0,5,0.9985,Phenix City,AL,USA,"{'latitude': 32.4709761, 'longitude': -85.0007..."
3,108702,5141711,HighSchool,2024,4,Ryan Williams,Saraland,Alabama,WR,72.5,165.0,5,0.9982,Saraland,AL,USA,"{'latitude': 30.820742, 'longitude': -88.07055..."
4,108703,5079767,HighSchool,2024,5,Justin Williams,Oak Ridge,Georgia,LB,74.0,205.0,5,0.9969,Conroe,TX,USA,"{'latitude': 30.3118769, 'longitude': -95.4560..."


In [4]:
df.columns.tolist()

['id',
 'athleteId',
 'recruitType',
 'year',
 'ranking',
 'name',
 'school',
 'committedTo',
 'position',
 'height',
 'weight',
 'stars',
 'rating',
 'city',
 'stateProvince',
 'country',
 'hometownInfo']

In [5]:
keep_cols = [
    "id",
    "name",
    "year",
    "position",
    "height",
    "weight",
    "stars",
    "rating",
    "ranking",
    "school",
    "committedTo",
    "city",
    "stateProvince"
]

hs_df = df[keep_cols].copy()

print(hs_df.shape)

(2548, 13)


In [6]:
hs_df.isna().sum()

id                 0
name               0
year               0
position           0
height             0
weight             1
stars              0
rating             0
ranking            0
school             0
committedTo      612
city               0
stateProvince      0
dtype: int64

In [9]:
hs_df["committedTo"] = hs_df["committedTo"].fillna("Uncommitted")

In [10]:
hs_df["committedTo"].isna().sum()

np.int64(0)

In [11]:
hs_df.to_csv("high_school_recruits_2024.csv", index=False)

In [12]:
import os
os.path.exists("high_school_recruits_2024.csv")

True

In [13]:
import time
import pandas as pd
import requests

years = list(range(2005, 2025))
all_rows = []

for y in years:
    r = requests.get(
        "https://api.collegefootballdata.com/recruiting/players",
        headers=headers,
        params={"year": y},
        timeout=30
    )
    r.raise_for_status()
    all_rows.append(pd.DataFrame(r.json()))
    time.sleep(0.25)

recruits_df = pd.concat(all_rows, ignore_index=True)

print(recruits_df.shape)

(59147, 17)


In [14]:
keep_cols = [
    "id",
    "name",
    "year",
    "position",
    "height",
    "weight",
    "stars",
    "rating",
    "ranking",
    "school",
    "committedTo",
    "city",
    "stateProvince"
]

hs_all = recruits_df[keep_cols].copy()
hs_all["committedTo"] = hs_all["committedTo"].fillna("Uncommitted")

print(hs_all.shape)

(59147, 13)


In [15]:
hs_all.to_csv("high_school_recruits_2005_2024.csv", index=False)

In [16]:
import os
os.path.exists("high_school_recruits_2005_2024.csv")


True

In [17]:
import pandas as pd

url = "https://www.sports-reference.com/cfb/years/2024-passing.html"
tables = pd.read_html(url)

print(len(tables))
print(tables[0].shape)

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1010)>

In [18]:
import certifi
import requests
import pandas as pd

url = "https://www.sports-reference.com/cfb/years/2024-passing.html"

html = requests.get(url, timeout=30, verify=certifi.where()).text
tables = pd.read_html(html)

print(len(tables))
print(tables[0].shape)

/var/folders/9b/n8r8r4xj2c591v5s8dycrwb80000gn/T/ipykernel_45109/3806447238.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html)


1
(511, 19)


In [19]:
tables[0].columns.tolist()

['Rk',
 'Player',
 'Team',
 'Conf',
 'G',
 'Cmp',
 'Att',
 'Cmp%',
 'Yds',
 'TD',
 'TD%',
 'Int',
 'Int%',
 'Y/A',
 'AY/A',
 'Y/C',
 'Y/G',
 'Rate',
 'Awards']

In [26]:
pass_2024 = tables[0].copy()

pass_2024 = pass_2024.drop(columns=["Rk"])
pass_2024["Player"] = pass_2024["Player"].str.replace("*", "", regex=False)

num_cols = ["G","Cmp","Att","Cmp%","Yds","TD","TD%","Int","Int%","Y/A","AY/A","Y/C","Y/G","Rate"]
pass_2024[num_cols] = pass_2024[num_cols].apply(pd.to_numeric, errors="coerce")

print(pass_2024.shape)
pass_2024.head()

(511, 18)


,Player,Team,Conf,G,Cmp,Att,Cmp%,Yds,TD,TD%,Int,Int%,Y/A,AY/A,Y/C,Y/G,Rate,Awards
0,Kyle McCord,Syracuse,ACC,13.0,391.0,592.0,66.0,4779.0,34.0,5.7,12.0,2.0,8.1,8.31,12.2,367.6,148.8,H-10
1,Cameron Ward,Miami (FL),ACC,13.0,305.0,454.0,67.2,4313.0,39.0,8.6,7.0,1.5,9.5,10.52,14.1,331.8,172.2,"H-4,AA"
2,Jaxson Dart,Ole Miss,SEC,13.0,276.0,398.0,69.3,4279.0,29.0,7.3,6.0,1.5,10.8,11.53,15.5,329.2,180.7,NaN
3,Shedeur Sanders,Colorado,Big 12,13.0,353.0,477.0,74.0,4134.0,37.0,7.8,10.0,2.1,8.7,9.27,11.7,318.0,168.2,H-8
4,Garrett Nussmeier,LSU,SEC,13.0,337.0,525.0,64.2,4052.0,29.0,5.5,12.0,2.3,7.7,7.79,12.0,311.7,142.7,NaN


In [27]:
pass_2024 = pass_2024.drop(columns=["G", "Awards"])

print(pass_2024.shape)

(511, 16)


In [28]:
pass_2024.head()

,Player,Team,Conf,Cmp,Att,Cmp%,Yds,TD,TD%,Int,Int%,Y/A,AY/A,Y/C,Y/G,Rate
0,Kyle McCord,Syracuse,ACC,391.0,592.0,66.0,4779.0,34.0,5.7,12.0,2.0,8.1,8.31,12.2,367.6,148.8
1,Cameron Ward,Miami (FL),ACC,305.0,454.0,67.2,4313.0,39.0,8.6,7.0,1.5,9.5,10.52,14.1,331.8,172.2
2,Jaxson Dart,Ole Miss,SEC,276.0,398.0,69.3,4279.0,29.0,7.3,6.0,1.5,10.8,11.53,15.5,329.2,180.7
3,Shedeur Sanders,Colorado,Big 12,353.0,477.0,74.0,4134.0,37.0,7.8,10.0,2.1,8.7,9.27,11.7,318.0,168.2
4,Garrett Nussmeier,LSU,SEC,337.0,525.0,64.2,4052.0,29.0,5.5,12.0,2.3,7.7,7.79,12.0,311.7,142.7


In [29]:
pass_2024 = pass_2024[pass_2024["Player"] != "League Average"].copy()

print(pass_2024.shape)

(510, 16)


In [30]:
pass_2024 = pass_2024.reset_index(drop=True)

In [31]:
pass_2024.head()

,Player,Team,Conf,Cmp,Att,Cmp%,Yds,TD,TD%,Int,Int%,Y/A,AY/A,Y/C,Y/G,Rate
0,Kyle McCord,Syracuse,ACC,391.0,592.0,66.0,4779.0,34.0,5.7,12.0,2.0,8.1,8.31,12.2,367.6,148.8
1,Cameron Ward,Miami (FL),ACC,305.0,454.0,67.2,4313.0,39.0,8.6,7.0,1.5,9.5,10.52,14.1,331.8,172.2
2,Jaxson Dart,Ole Miss,SEC,276.0,398.0,69.3,4279.0,29.0,7.3,6.0,1.5,10.8,11.53,15.5,329.2,180.7
3,Shedeur Sanders,Colorado,Big 12,353.0,477.0,74.0,4134.0,37.0,7.8,10.0,2.1,8.7,9.27,11.7,318.0,168.2
4,Garrett Nussmeier,LSU,SEC,337.0,525.0,64.2,4052.0,29.0,5.5,12.0,2.3,7.7,7.79,12.0,311.7,142.7


In [32]:
pass_2024.to_csv("college_passing_2024_clean.csv", index=False)

In [33]:
from io import StringIO
import certifi
import requests
import pandas as pd

def fetch_passing_year(year: int) -> pd.DataFrame:
    url = f"https://www.sports-reference.com/cfb/years/{year}-passing.html"
    html = requests.get(url, timeout=30, verify=certifi.where()).text

    df = pd.read_html(StringIO(html))[0].copy()

    df = df.drop(columns=["Rk", "G", "Awards"], errors="ignore")
    df["Player"] = df["Player"].astype(str).str.replace("*", "", regex=False)
    df = df[df["Player"] != "League Average"].copy()

    num_cols = ["Cmp","Att","Cmp%","Yds","TD","TD%","Int","Int%","Y/A","AY/A","Y/C","Y/G","Rate"]
    for c in num_cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")

    df["season"] = year
    return df.reset_index(drop=True)

test = fetch_passing_year(2024)
print(test.shape)

(510, 17)


In [34]:
import time
import pandas as pd

years = list(range(2005, 2025))
frames = []

for y in years:
    frames.append(fetch_passing_year(y))
    time.sleep(0.25)

passing_all = pd.concat(frames, ignore_index=True)

print(passing_all.shape)

(9784, 17)


In [35]:
passing_all.head

<bound method NDFrame.head of                Player                Team      Conf    Cmp    Att   Cmp%  \
0        Colt Brennan              Hawaii       WAC  350.0  515.0   68.0   
1         Cody Hodges          Texas Tech    Big 12  353.0  531.0   66.5   
2         Brady Quinn          Notre Dame       Ind  292.0  450.0   64.9   
3        Matt Leinart                 USC    Pac-10  283.0  431.0   65.7   
4           John Beck                 BYU       MWC  331.0  513.0   64.5   
...               ...                 ...       ...    ...    ...    ...   
9779     Wilson Berry            Kentucky       SEC    0.0    1.0    0.0   
9780  Xavier Restrepo          Miami (FL)       ACC    0.0    1.0    0.0   
9781      Zion Turner  Jacksonville State      CUSA    0.0    1.0    0.0   
9782      Peter Moore       Virginia Tech       ACC    1.0    1.0  100.0   
9783      Nate Rogers                 UAB  American    1.0    1.0  100.0   

         Yds    TD  TD%   Int   Int%  Y/A   AY/A   Y/C   

In [36]:
passing_all.to_csv("college_passing_2005_2024_clean.csv", index=False)

In [37]:
from io import StringIO
import certifi
import requests
import pandas as pd

url = "https://www.sports-reference.com/cfb/years/2024-rushing.html"
html = requests.get(url, timeout=30, verify=certifi.where()).text

rush_tables = pd.read_html(StringIO(html))
rush_2024_raw = rush_tables[0]

print(len(rush_tables))
print(rush_2024_raw.shape)

1
(1659, 20)


In [39]:
rush_2024_raw.head

<bound method NDFrame.head of      Unnamed: 0_level_0 Unnamed: 1_level_0  Unnamed: 2_level_0  \
                     Rk             Player                Team   
0                   1.0      Ashton Jeanty         Boise State   
1                   2.0   Cameron Skattebo       Arizona State   
2                   3.0   Omarion Hampton*      North Carolina   
3                   4.0      Bryson Daily*                Army   
4                   5.0       Tre Stewart*  Jacksonville State   
...                 ...                ...                 ...   
1654             1655.0      Hayden Wolff*    Western Michigan   
1655             1656.0     Deshawn Purdie           Charlotte   
1656             1657.0      Evan Bullock*      Louisiana Tech   
1657             1658.0       Mikey Keene*        Fresno State   
1658                NaN     League Average                 NaN   

     Unnamed: 3_level_0 Unnamed: 4_level_0 Rushing                            \
                   Conf        

In [40]:
rush_2024_raw.columns.tolist()

[('Unnamed: 0_level_0', 'Rk'),
 ('Unnamed: 1_level_0', 'Player'),
 ('Unnamed: 2_level_0', 'Team'),
 ('Unnamed: 3_level_0', 'Conf'),
 ('Unnamed: 4_level_0', 'G'),
 ('Rushing', 'Att'),
 ('Rushing', 'Yds'),
 ('Rushing', 'Y/A'),
 ('Rushing', 'TD'),
 ('Rushing', 'Y/G'),
 ('Receiving', 'Rec'),
 ('Receiving', 'Yds'),
 ('Receiving', 'Y/R'),
 ('Receiving', 'TD'),
 ('Receiving', 'Y/G'),
 ('Scrimmage', 'Plays'),
 ('Scrimmage', 'Yds'),
 ('Scrimmage', 'Avg'),
 ('Scrimmage', 'TD'),
 ('Unnamed: 19_level_0', 'Awards')]

In [41]:
rush_2024 = rush_2024_raw.copy()

rush_2024.columns = [
    f"{a}_{b}" if "Unnamed" not in a else b
    for a, b in rush_2024.columns
]

rush_2024.columns.tolist()

['Rk',
 'Player',
 'Team',
 'Conf',
 'G',
 'Rushing_Att',
 'Rushing_Yds',
 'Rushing_Y/A',
 'Rushing_TD',
 'Rushing_Y/G',
 'Receiving_Rec',
 'Receiving_Yds',
 'Receiving_Y/R',
 'Receiving_TD',
 'Receiving_Y/G',
 'Scrimmage_Plays',
 'Scrimmage_Yds',
 'Scrimmage_Avg',
 'Scrimmage_TD',
 'Awards']

In [42]:
rush_2024 = rush_2024.drop(columns=["Rk", "G", "Awards"])

In [43]:
rush_2024.shape

(1659, 17)

In [44]:
rush_2024["Player"] = rush_2024["Player"].str.replace("*", "", regex=False)

num_cols = [
    "Rushing_Att","Rushing_Yds","Rushing_Y/A","Rushing_TD","Rushing_Y/G",
    "Receiving_Rec","Receiving_Yds","Receiving_Y/R","Receiving_TD","Receiving_Y/G",
    "Scrimmage_Plays","Scrimmage_Yds","Scrimmage_Avg","Scrimmage_TD"
]

rush_2024[num_cols] = rush_2024[num_cols].apply(pd.to_numeric, errors="coerce")

rush_2024.isna().sum().sum()

np.int64(614)

In [45]:
rush_2024[num_cols] = rush_2024[num_cols].fillna(0)

In [46]:
rush_2024.isna().sum().sum()

np.int64(2)

In [47]:
rush_2024.isna().sum()

Player             0
Team               1
Conf               1
Rushing_Att        0
Rushing_Yds        0
Rushing_Y/A        0
Rushing_TD         0
Rushing_Y/G        0
Receiving_Rec      0
Receiving_Yds      0
Receiving_Y/R      0
Receiving_TD       0
Receiving_Y/G      0
Scrimmage_Plays    0
Scrimmage_Yds      0
Scrimmage_Avg      0
Scrimmage_TD       0
dtype: int64

In [48]:
rush_2024["Team"] = rush_2024["Team"].fillna("Unknown")
rush_2024["Conf"] = rush_2024["Conf"].fillna("Unknown")

rush_2024.isna().sum().sum()

np.int64(0)

In [49]:
rush_2024.to_csv("college_rushing_receiving_2024_clean.csv", index=False)

In [50]:
from io import StringIO
import certifi
import requests
import pandas as pd

def fetch_rushing_year(year: int) -> pd.DataFrame:
    url = f"https://www.sports-reference.com/cfb/years/{year}-rushing.html"
    html = requests.get(url, timeout=30, verify=certifi.where()).text

    df = pd.read_html(StringIO(html))[0].copy()

    # Flatten multi-level columns
    df.columns = [
        f"{a}_{b}" if "Unnamed" not in a else b
        for a, b in df.columns
    ]

    # Drop non-essential columns
    df = df.drop(columns=["Rk", "G", "Awards"], errors="ignore")

    # Clean player names
    df["Player"] = df["Player"].astype(str).str.replace("*", "", regex=False)

    # Remove non-player rows
    df = df[df["Player"] != "League Average"].copy()

    # Numeric columns
    num_cols = [
        "Rushing_Att","Rushing_Yds","Rushing_Y/A","Rushing_TD","Rushing_Y/G",
        "Receiving_Rec","Receiving_Yds","Receiving_Y/R","Receiving_TD","Receiving_Y/G",
        "Scrimmage_Plays","Scrimmage_Yds","Scrimmage_Avg","Scrimmage_TD"
    ]
    for c in num_cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0)

    # Fill identity fields
    df["Team"] = df["Team"].fillna("Unknown")
    df["Conf"] = df["Conf"].fillna("Unknown")

    df["season"] = year
    return df.reset_index(drop=True)

test_rush = fetch_rushing_year(2024)
print(test_rush.shape)

(1658, 18)


In [51]:
import time
import pandas as pd

years = list(range(2005, 2025))
frames = []

for y in years:
    frames.append(fetch_rushing_year(y))
    time.sleep(0.25)

rushing_all = pd.concat(frames, ignore_index=True)

print(rushing_all.shape)

(30053, 18)


In [52]:
rushing_all.head

<bound method NDFrame.head of                   Player               Team      Conf  Rushing_Att  \
0      DeAngelo Williams            Memphis      CUSA        310.0   
1        Jerome Harrison   Washington State    Pac-10        308.0   
2            Reggie Bush                USC    Pac-10        200.0   
3          Brian Calhoun          Wisconsin   Big Ten        348.0   
4          Garrett Wolfe  Northern Illinois       MAC        242.0   
...                  ...                ...       ...          ...   
30048     Connor Bazelak      Bowling Green       MAC         48.0   
30049       Hayden Wolff   Western Michigan       MAC         39.0   
30050     Deshawn Purdie          Charlotte  American         41.0   
30051       Evan Bullock     Louisiana Tech      CUSA         63.0   
30052        Mikey Keene       Fresno State       MWC         47.0   

       Rushing_Yds  Rushing_Y/A  Rushing_TD  Rushing_Y/G  Receiving_Rec  \
0           1964.0          6.3        18.0        178

In [53]:
rushing_all.to_csv("college_rushing_receiving_2005_2024_clean.csv", index=False)

In [54]:
qb_passing = passing_all[passing_all["Att"] > 0].copy()

print(qb_passing.shape)

(9784, 17)


In [55]:
qb_keys = qb_passing[["Player", "season", "Team"]].drop_duplicates()

qb_rushing = rushing_all.merge(
    qb_keys,
    on=["Player", "season", "Team"],
    how="inner"
)

print(qb_rushing.shape)

(8426, 18)


In [56]:
qb_full = qb_passing.merge(
    qb_rushing.drop(columns=["Conf"]),  # avoid duplicate Conf
    on=["Player", "season", "Team"],
    how="left",
    suffixes=("", "_rush")
)

print(qb_full.shape)

(9788, 31)


In [57]:
rush_cols = [c for c in qb_full.columns if c.startswith(("Rushing_", "Receiving_", "Scrimmage_"))]

qb_full[rush_cols] = qb_full[rush_cols].fillna(0)

qb_full[rush_cols].isna().sum().sum()

np.int64(0)

In [58]:
qb_full = qb_full.sort_values(["season", "Player", "Team"]).drop_duplicates(
    subset=["Player", "season", "Team"],
    keep="first"
).reset_index(drop=True)

print(qb_full.shape)

(9777, 31)


In [59]:
qb_full.to_csv("college_qb_passing_plus_rushing_2005_2024_clean.csv", index=False)

In [60]:
rb_base = rushing_all[rushing_all["Rushing_Att"] > 0].copy()

print(rb_base.shape)

(30053, 18)


In [61]:
qb_keys = qb_full[["Player", "season", "Team"]].drop_duplicates()

rb_only = rb_base.merge(
    qb_keys,
    on=["Player", "season", "Team"],
    how="left",
    indicator=True
)

rb_only = rb_only[rb_only["_merge"] == "left_only"].drop(columns=["_merge"])

print(rb_only.shape)

(21627, 18)


In [62]:
rb_only.to_csv("college_rb_rushing_receiving_2005_2024_clean.csv", index=False)

In [63]:
import re

def norm_name(s: str) -> str:
    if s is None:
        return ""
    s = str(s).lower().strip()
    s = re.sub(r"[^\w\s]", "", s)              # remove punctuation
    s = re.sub(r"\b(jr|sr|ii|iii|iv|v)\b", "", s)  # remove suffixes
    s = re.sub(r"\s+", " ", s).strip()         # collapse spaces
    return s

hs_all["name_key"] = hs_all["name"].apply(norm_name)
qb_full["name_key"] = qb_full["Player"].apply(norm_name)
rb_only["name_key"] = rb_only["Player"].apply(norm_name)

hs_all["state_key"] = hs_all["stateProvince"].astype(str).str.upper().str.strip()

In [64]:
hs_all["name_key"].head().tolist()

['eugene monroe',
 'patrick turner',
 'mark sanchez',
 'derrick williams',
 'fred rouse']

In [65]:
hs_qb_name_matches = hs_all.merge(
    qb_full[["name_key"]].drop_duplicates(),
    on="name_key",
    how="inner"
)

print(hs_qb_name_matches.shape)

(3866, 15)


In [66]:
matched = hs_qb_name_matches["id"].nunique()
total = hs_all["id"].nunique()
print(matched, total, round(matched / total, 4))

3866 59147 0.0654


In [67]:
hs_qb_window = hs_all.merge(
    qb_full[["name_key", "season"]],
    on="name_key",
    how="inner"
)

hs_qb_window = hs_qb_window[
    (hs_qb_window["season"] >= hs_qb_window["year"]) &
    (hs_qb_window["season"] <= hs_qb_window["year"] + 6)
]

print(hs_qb_window.shape)

(7318, 16)


In [68]:
matched = hs_qb_window["id"].nunique()
total = hs_all["id"].nunique()
print(matched, total, round(matched / total, 4))

3358 59147 0.0568


In [70]:
import re

def norm_school(s: str) -> str:
    s = "" if s is None else str(s).lower().strip()
    s = re.sub(r"[^\w\s]", "", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

hs_qb_window["committed_key"] = hs_qb_window["committedTo"].apply(norm_school)
hs_qb_window["team_key"] = hs_qb_window["Team"].apply(norm_school)

hs_qb_committed_match = hs_qb_window[
    (hs_qb_window["committedTo"] == "Uncommitted") |
    (hs_qb_window["committed_key"] == hs_qb_window["team_key"])
].copy()

print(hs_qb_committed_match.shape)

KeyError: 'Team'

In [71]:
hs_qb_window = hs_all.merge(
    qb_full[["name_key", "season", "Team"]],
    on="name_key",
    how="inner"
)

hs_qb_window = hs_qb_window[
    (hs_qb_window["season"] >= hs_qb_window["year"]) &
    (hs_qb_window["season"] <= hs_qb_window["year"] + 6)
].copy()

print(hs_qb_window.shape)

(7318, 17)


In [72]:
import re

def norm_school(s: str) -> str:
    s = "" if s is None else str(s).lower().strip()
    s = re.sub(r"[^\w\s]", "", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

hs_qb_window["committed_key"] = hs_qb_window["committedTo"].apply(norm_school)
hs_qb_window["team_key"] = hs_qb_window["Team"].apply(norm_school)

hs_qb_committed_match = hs_qb_window[
    (hs_qb_window["committedTo"] == "Uncommitted") |
    (hs_qb_window["committed_key"] == hs_qb_window["team_key"])
].copy()

print(hs_qb_committed_match.shape)

(5544, 19)


In [73]:
matched = hs_qb_committed_match["id"].nunique()
total = hs_all["id"].nunique()
print(matched, total, round(matched / total, 4))

2700 59147 0.0456


In [74]:
hs_rb_window = hs_all.merge(
    rb_only[["name_key", "season", "Team"]],
    on="name_key",
    how="inner"
)

hs_rb_window = hs_rb_window[
    (hs_rb_window["season"] >= hs_rb_window["year"]) &
    (hs_rb_window["season"] <= hs_rb_window["year"] + 6)
].copy()

print(hs_rb_window.shape)

(15502, 17)


In [75]:
hs_rb_window["committed_key"] = hs_rb_window["committedTo"].apply(norm_school)
hs_rb_window["team_key"] = hs_rb_window["Team"].apply(norm_school)

hs_rb_committed_match = hs_rb_window[
    (hs_rb_window["committedTo"] == "Uncommitted") |
    (hs_rb_window["committed_key"] == hs_rb_window["team_key"])
].copy()

print(hs_rb_committed_match.shape)

(12145, 19)


In [76]:
matched = hs_rb_committed_match["id"].nunique()
total = hs_all["id"].nunique()
print(matched, total, round(matched / total, 4))

5928 59147 0.1002


In [77]:
hs_rb_best = hs_rb_committed_match.sort_values(["id", "season"]).drop_duplicates(
    subset=["id"],
    keep="first"
).reset_index(drop=True)

print(hs_rb_best.shape)

(5928, 19)


In [78]:
hs_qb_best = hs_qb_committed_match.sort_values(["id", "season"]).drop_duplicates(
    subset=["id"],
    keep="first"
).reset_index(drop=True)

print(hs_qb_best.shape)

(2700, 19)


In [79]:
hs_qb_best.to_csv("hs_to_college_qb_bestmatch.csv", index=False)
hs_rb_best.to_csv("hs_to_college_rb_bestmatch.csv", index=False)

In [1]:
hs_qb_best[["name", "year", "committedTo", "season", "Team", "position", "stars", "rating"]].head()

NameError: name 'hs_qb_best' is not defined

In [2]:
import pandas as pd

hs_qb_best = pd.read_csv("hs_to_college_qb_bestmatch.csv")

hs_qb_best[["name", "year", "committedTo", "season", "Team", "position", "stars", "rating"]].head()

,name,year,committedTo,season,Team,position,stars,rating
0,DJ Lagway,2024,Florida,2024,Florida,QB,5,0.9964
1,Dylan Raiola,2024,Nebraska,2024,Nebraska,QB,5,0.9926
2,Taylor Tatum,2024,Oklahoma,2024,Oklahoma,RB,4,0.9823
3,Luke Kromenhoek,2024,Florida State,2024,Florida State,QB,4,0.9718
4,Ethan Grunkemeyer,2024,Penn State,2024,Penn State,QB,4,0.9505


In [3]:
hs_qb_best[["name", "year", "committedTo", "season", "Team", "position", "stars", "rating"]].head()

,name,year,committedTo,season,Team,position,stars,rating
0,DJ Lagway,2024,Florida,2024,Florida,QB,5,0.9964
1,Dylan Raiola,2024,Nebraska,2024,Nebraska,QB,5,0.9926
2,Taylor Tatum,2024,Oklahoma,2024,Oklahoma,RB,4,0.9823
3,Luke Kromenhoek,2024,Florida State,2024,Florida State,QB,4,0.9718
4,Ethan Grunkemeyer,2024,Penn State,2024,Penn State,QB,4,0.9505


In [4]:
hs_rb_best = pd.read_csv("hs_to_college_rb_bestmatch.csv")

hs_rb_best[["name", "year", "committedTo", "season", "Team", "position", "stars", "rating"]].head()

,name,year,committedTo,season,Team,position,stars,rating
0,Jeremiah Smith,2024,Ohio State,2024,Ohio State,WR,5,0.9997
1,Ryan Williams,2024,Alabama,2024,Alabama,WR,5,0.9982
2,Terry Bussey,2024,Texas A&M,2024,Texas A&M,ATH,5,0.9945
3,Ryan Wingo,2024,Texas,2024,Texas,WR,4,0.9819
4,Nate Frazier,2024,Georgia,2024,Georgia,RB,4,0.9777


In [5]:
qb_stats = pd.read_csv("college_qb_passing_plus_rushing_2005_2024_clean.csv")

qb_stats[["Player", "season", "Team", "Att", "Yds", "TD", "Rushing_Yds", "Scrimmage_TD"]].head()

,Player,season,Team,Att,Yds,TD,Rushing_Yds,Scrimmage_TD
0,A.J. Harris,2005,Northern Illinois,1.0,0.0,0.0,351.0,5.0
1,Aaron Perez,2005,UCLA,1.0,0.0,0.0,0.0,0.0
2,Adam Austin,2005,Arizona,5.0,32.0,0.0,-15.0,0.0
3,Adam Barmann,2005,Kansas,30.0,118.0,0.0,-2.0,1.0
4,Adam Bednarik,2005,West Virginia,75.0,532.0,4.0,170.0,1.0


In [7]:
hs_qb_final = hs_qb_best.merge(
    qb_stats,
    left_on=["name_key", "season", "Team"],
    right_on=["name_key", "season", "Team"],
    how="left",
    suffixes=("", "_college")
)

print(hs_qb_final.shape)

KeyError: 'name_key'

In [8]:
import re

def norm_name(s: str) -> str:
    if s is None:
        return ""
    s = str(s).lower().strip()
    s = re.sub(r"[^\w\s]", "", s)
    s = re.sub(r"\b(jr|sr|ii|iii|iv|v)\b", "", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

hs_qb_best["name_key"] = hs_qb_best["name"].apply(norm_name)
qb_stats["name_key"] = qb_stats["Player"].apply(norm_name)

print(hs_qb_best[["name", "name_key"]].head(3))
print(qb_stats[["Player", "name_key"]].head(3))

           name      name_key
0     DJ Lagway     dj lagway
1  Dylan Raiola  dylan raiola
2  Taylor Tatum  taylor tatum
        Player     name_key
0  A.J. Harris    aj harris
1  Aaron Perez  aaron perez
2  Adam Austin  adam austin


In [9]:
hs_qb_final = hs_qb_best.merge(
    qb_stats,
    on=["name_key", "season", "Team"],
    how="left"
)

print(hs_qb_final.shape)

(2701, 48)


In [10]:
print(hs_qb_final["Att"].notna().mean())

1.0


In [11]:
rb_stats = pd.read_csv("college_rb_rushing_receiving_2005_2024_clean.csv")

rb_stats["name_key"] = rb_stats["Player"].apply(norm_name)
hs_rb_best["name_key"] = hs_rb_best["name"].apply(norm_name)

hs_rb_final = hs_rb_best.merge(
    rb_stats,
    on=["name_key", "season", "Team"],
    how="left"
)

print(hs_rb_final.shape)
print(hs_rb_final["Rushing_Att"].notna().mean())

(5954, 35)
1.0


In [12]:
hs_qb_final.to_csv("final_hs_to_college_qb_dataset.csv", index=False)
hs_rb_final.to_csv("final_hs_to_college_rb_dataset.csv", index=False)

In [13]:
import pandas as pd
df_check = pd.read_csv("final_hs_to_college_qb_dataset.csv")
df_check.head()

,id,name,year,position,height,weight,stars,rating,ranking,school,...,Rushing_Y/G,Receiving_Rec,Receiving_Yds,Receiving_Y/R,Receiving_TD,Receiving_Y/G,Scrimmage_Plays,Scrimmage_Yds,Scrimmage_Avg,Scrimmage_TD
0,108705,DJ Lagway,2024,QB,74.0,225.0,5,0.9964,7,Willis,...,8.4,0.0,0.0,0.0,0.0,0.0,51.0,101.0,2.0,0.0
1,108719,Dylan Raiola,2024,QB,75.0,230.0,5,0.9926,21,Buford,...,-5.0,0.0,0.0,0.0,0.0,0.0,50.0,-65.0,-1.3,0.0
2,108736,Taylor Tatum,2024,RB,70.5,205.0,4,0.9823,38,Longview,...,25.3,5.0,41.0,8.2,1.0,3.7,61.0,319.0,5.2,4.0
3,108757,Luke Kromenhoek,2024,QB,76.0,185.0,4,0.9718,59,Benedictine Military School,...,18.8,0.0,0.0,0.0,0.0,0.0,63.0,113.0,1.8,0.0
4,108803,Ethan Grunkemeyer,2024,QB,74.0,190.0,4,0.9505,105,Olentangy,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
hs_qb_final.columns.tolist()

['id',
 'name',
 'year',
 'position',
 'height',
 'weight',
 'stars',
 'rating',
 'ranking',
 'school',
 'committedTo',
 'city',
 'stateProvince',
 'name_key',
 'state_key',
 'season',
 'Team',
 'committed_key',
 'team_key',
 'Player',
 'Conf',
 'Cmp',
 'Att',
 'Cmp%',
 'Yds',
 'TD',
 'TD%',
 'Int',
 'Int%',
 'Y/A',
 'AY/A',
 'Y/C',
 'Y/G',
 'Rate',
 'Rushing_Att',
 'Rushing_Yds',
 'Rushing_Y/A',
 'Rushing_TD',
 'Rushing_Y/G',
 'Receiving_Rec',
 'Receiving_Yds',
 'Receiving_Y/R',
 'Receiving_TD',
 'Receiving_Y/G',
 'Scrimmage_Plays',
 'Scrimmage_Yds',
 'Scrimmage_Avg',
 'Scrimmage_TD']

In [15]:
import requests
import pandas as pd
from pathlib import Path

API_KEY = Path("API KEY.txt").read_text().strip()
headers = {"Authorization": f"Bearer {API_KEY}", "User-Agent": "Mozilla/5.0"}

url = "https://api.collegefootballdata.com/combine"
r = requests.get(url, headers=headers, timeout=30)
r.raise_for_status()

combine_df = pd.DataFrame(r.json())

print(combine_df.shape)

JSONDecodeError: Expecting value: line 2 column 1 (char 1)

In [16]:
print("status:", r.status_code)
print(r.text[:500])

status: 200

<!-- HTML for static distribution bundle build -->
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  
  <title>Swagger UI</title>
  <link rel="stylesheet" type="text/css" href="./swagger-ui.css" >
  <link rel="icon" type="image/png" href="./favicon-32x32.png" sizes="32x32" /><link rel="icon" type="image/png" href="./favicon-16x16.png" sizes="16x16" />
  <style>
    html
    {
      box-sizing: border-box;
      overflow: -moz-scrollbars-vertical;
      overflow-y: scroll;
    }


In [17]:
import pandas as pd
import requests
from io import StringIO

url = "https://www.pro-football-reference.com/years/2024/draft.htm"
html = requests.get(url, headers={"User-Agent": "Mozilla/5.0"}, timeout=30).text

draft_2024 = pd.read_html(StringIO(html), attrs={"id": "drafts"})[0]

print(draft_2024.shape)

(263, 29)


In [18]:
draft_2024.columns.tolist()

[('Unnamed: 0_level_0', 'Rnd'),
 ('Unnamed: 1_level_0', 'Pick'),
 ('Unnamed: 2_level_0', 'Tm'),
 ('Unnamed: 3_level_0', 'Player'),
 ('Unnamed: 4_level_0', 'Pos'),
 ('Unnamed: 5_level_0', 'Age'),
 ('Unnamed: 6_level_0', 'To'),
 ('Misc', 'AP1'),
 ('Misc', 'PB'),
 ('Unnamed: 9_level_0', 'St'),
 ('Approx Val', 'wAV'),
 ('Approx Val', 'DrAV'),
 ('Unnamed: 12_level_0', 'G'),
 ('Passing', 'Cmp'),
 ('Passing', 'Att'),
 ('Passing', 'Yds'),
 ('Passing', 'TD'),
 ('Passing', 'Int'),
 ('Rushing', 'Att'),
 ('Rushing', 'Yds'),
 ('Rushing', 'TD'),
 ('Receiving', 'Rec'),
 ('Receiving', 'Yds'),
 ('Receiving', 'TD'),
 ('Unnamed: 24_level_0', 'Solo'),
 ('Unnamed: 25_level_0', 'Int'),
 ('Unnamed: 26_level_0', 'Sk'),
 ('Unnamed: 27_level_0', 'College/Univ'),
 ('Unnamed: 28_level_0', 'Unnamed: 28_level_1')]

In [19]:
draft_2024_clean = draft_2024.copy()

draft_2024_clean.columns = [
    f"{a}_{b}" if "Unnamed" not in a else b
    for a, b in draft_2024_clean.columns
]

draft_2024_clean.columns.tolist()

['Rnd',
 'Pick',
 'Tm',
 'Player',
 'Pos',
 'Age',
 'To',
 'Misc_AP1',
 'Misc_PB',
 'St',
 'Approx Val_wAV',
 'Approx Val_DrAV',
 'G',
 'Passing_Cmp',
 'Passing_Att',
 'Passing_Yds',
 'Passing_TD',
 'Passing_Int',
 'Rushing_Att',
 'Rushing_Yds',
 'Rushing_TD',
 'Receiving_Rec',
 'Receiving_Yds',
 'Receiving_TD',
 'Solo',
 'Int',
 'Sk',
 'College/Univ',
 'Unnamed: 28_level_1']

In [20]:
import pandas as pd

nfl_draft_2024 = draft_2024_clean.drop(columns=["Unnamed: 28_level_1"], errors="ignore").copy()

# Clean player names (remove Hall-of-Fame markers, asterisks, etc.)
nfl_draft_2024["Player"] = (
    nfl_draft_2024["Player"]
    .astype(str)
    .str.replace("*", "", regex=False)
    .str.replace("+", "", regex=False)
    .str.strip()
)

# Convert key numeric columns
num_cols = [
    "Rnd","Pick","Age","To","St","G",
    "Passing_Cmp","Passing_Att","Passing_Yds","Passing_TD","Passing_Int",
    "Rushing_Att","Rushing_Yds","Rushing_TD",
    "Receiving_Rec","Receiving_Yds","Receiving_TD",
    "Solo","Int","Sk",
    "Misc_AP1","Misc_PB",
    "Approx Val_wAV","Approx Val_DrAV"
]
for c in num_cols:
    if c in nfl_draft_2024.columns:
        nfl_draft_2024[c] = pd.to_numeric(nfl_draft_2024[c], errors="coerce")

nfl_draft_2024["draft_year"] = 2024

print(nfl_draft_2024.shape)

(263, 29)


In [21]:
from io import StringIO
import requests
import pandas as pd

def fetch_nfl_draft_year(year: int) -> pd.DataFrame:
    url = f"https://www.pro-football-reference.com/years/{year}/draft.htm"
    html = requests.get(url, headers={"User-Agent": "Mozilla/5.0"}, timeout=30).text

    df = pd.read_html(StringIO(html), attrs={"id": "drafts"})[0].copy()

    # Flatten columns
    df.columns = [f"{a}_{b}" if "Unnamed" not in a else b for a, b in df.columns]

    # Drop junk column
    df = df.drop(columns=["Unnamed: 28_level_1"], errors="ignore").copy()

    # Clean names
    df["Player"] = (
        df["Player"].astype(str)
        .str.replace("*", "", regex=False)
        .str.replace("+", "", regex=False)
        .str.strip()
    )

    # Numeric coercion
    num_cols = [
        "Rnd","Pick","Age","To","St","G",
        "Passing_Cmp","Passing_Att","Passing_Yds","Passing_TD","Passing_Int",
        "Rushing_Att","Rushing_Yds","Rushing_TD",
        "Receiving_Rec","Receiving_Yds","Receiving_TD",
        "Solo","Int","Sk",
        "Misc_AP1","Misc_PB",
        "Approx Val_wAV","Approx Val_DrAV"
    ]
    for c in num_cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")

    df["draft_year"] = year
    return df

test_draft = fetch_nfl_draft_year(2024)
print(test_draft.shape)

(263, 29)


In [22]:
import time
import pandas as pd

years = list(range(2005, 2025))
draft_frames = []

for y in years:
    draft_frames.append(fetch_nfl_draft_year(y))
    time.sleep(0.25)

nfl_draft_all = pd.concat(draft_frames, ignore_index=True)

print(nfl_draft_all.shape)

(5229, 29)


In [23]:
nfl_draft_all.to_csv("nfl_draft_2005_2024_clean.csv", index=False)